In [2]:
import pandas as pd
import numpy as np

In [3]:
conf_data = pd.read_csv('../conflict_data_mmr.csv')
conf_data.head()

,id,relid,year,active_year,code_status,type_of_violence,conflict_dset_id,conflict_new_id,conflict_name,dyad_dset_id,...,date_end,deaths_a,deaths_b,deaths_civilians,deaths_unknown,best,high,low,gwnoa,gwnob
0,NaN,NaN,#date+year,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,#date+end,NaN,NaN,NaN,NaN,#affected+killed,NaN,NaN,NaN,NaN
1,71686.0,MYA-1989-1-306-35,1989,1.0,Clear,1.0,221.0,221.0,Myanmar (Burma): Karen,428.0,...,1989-12-31 00:00:00.000,0.0,0.0,0.0,3.0,3,0.0,0.0,775.0,NaN
2,500136.0,MYA-1989-1-306-1.2,1989,1.0,Clear,1.0,221.0,221.0,Myanmar (Burma): Karen,428.0,...,1989-02-21 00:00:00.000,0.0,0.0,4.0,0.0,4,4.0,4.0,775.0,NaN
3,49166.0,MYA-1989-1-306-1.1,1989,1.0,Clear,1.0,221.0,221.0,Myanmar (Burma): Karen,428.0,...,1989-02-21 00:00:00.000,16.0,0.0,0.0,0.0,16,16.0,16.0,775.0,NaN
4,49163.0,MYA-1989-1-306-13,1989,1.0,Clear,1.0,221.0,221.0,Myanmar (Burma): Karen,428.0,...,1989-04-20 00:00:00.000,161.0,466.0,0.0,0.0,627,761.0,177.0,775.0,NaN


In [4]:
conf_data['total_deaths'] = conf_data[['deaths_a', 'deaths_b', 'deaths_civilians', 'deaths_unknown']].sum(axis=1)
conf_data.head()

opp_factions = ['KIO', 'KNU', 'ULA', 'PSLF', 'CNF', 'KNPP', 'ARSA', 'SSPP', 'MNDAA', 'Civilians', 'RCSS', 'NUG']

#group by side_b and year, summing total deaths
grouped_data = conf_data.groupby(['side_b', 'year'])['total_deaths'].sum().reset_index()
#convert year to integer

#filter to only include rows where side_b is in opp_factions
grouped_data = grouped_data[grouped_data['side_b'].isin(opp_factions)]
grouped_data['year'] = grouped_data['year'].astype(int)
#add data for all years for every faction, filling missing years with 0 deaths
all_years = pd.DataFrame({'year': range(grouped_data['year'].min(), grouped_data['year'].max() + 1)})
all_factions = pd.DataFrame({'side_b': opp_factions})
all_combinations = all_factions.merge(all_years, how='cross')
grouped_data = all_combinations.merge(grouped_data, on=['side_b', 'year'], how='left').fillna(0)
grouped_data['total_deaths'] = grouped_data['total_deaths'].astype(int)


#calculate cumulative sum of total_deaths for each side_b
grouped_data['cumulative_deaths'] = grouped_data.groupby('side_b')['total_deaths'].cumsum()
grouped_data.head()

,side_b,year,total_deaths,cumulative_deaths
0,KIO,1989,194,194
1,KIO,1990,25,219
2,KIO,1991,25,244
3,KIO,1992,25,269
4,KIO,1993,0,269


In [5]:
grouped_data.to_csv('timeseries_mmr.csv', index=False)

## ACLED

In [7]:
acled_data= pd.read_excel('../apac_aggregated.xlsx')
acled_data.head()

,WEEK,REGION,COUNTRY,ADMIN1,EVENT_TYPE,SUB_EVENT_TYPE,EVENTS,FATALITIES,POPULATION_EXPOSURE,DISORDER_TYPE,ID,CENTROID_LATITUDE,CENTROID_LONGITUDE
0,2016-12-31,Caucasus and Central Asia,Afghanistan,Badakhshan,Battles,Armed clash,6,15,113469.0,Political violence,1,36.966,73.3417
1,2017-01-07,Caucasus and Central Asia,Afghanistan,Badakhshan,Battles,Armed clash,5,28,155849.0,Political violence,1,36.966,73.3417
2,2017-01-21,Caucasus and Central Asia,Afghanistan,Badakhshan,Battles,Armed clash,1,4,38453.0,Political violence,1,36.966,73.3417
3,2017-02-04,Caucasus and Central Asia,Afghanistan,Badakhshan,Battles,Armed clash,1,4,1308.0,Political violence,1,36.966,73.3417
4,2017-02-11,Caucasus and Central Asia,Afghanistan,Badakhshan,Battles,Armed clash,4,41,143199.0,Political violence,1,36.966,73.3417


In [16]:
mmr_data = acled_data[acled_data['COUNTRY'] == 'Myanmar']

#total political fatalities until 2023
pol_data = mmr_data[mmr_data['DISORDER_TYPE'] == 'Political violence']
pol_data = pol_data[pol_data['FATALITIES'] > 0]

pol_data['YEAR'] = pd.to_datetime(pol_data['WEEK']).dt.year
pol_data = pol_data[pol_data['YEAR'] <= 2023]
total_fatalities = pol_data['FATALITIES'].sum()
print("Total political fatalities in Myanmar until 2023:", total_fatalities)


Total political fatalities in Myanmar until 2023: 65619
